In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from matplotlib import patches as patches
from tensorflow.keras.preprocessing import image_dataset_from_directory#,image
from sklearn.model_selection import train_test_split
import glob
import cv2 as cv
from statistics import mean

## Declaración de variables

In [2]:
lado_mapa = 5
periodo = 500
learning_rate = 0.01

num_entradas = 1
num_datos = 100
dimensiones=[100,100]

# Calcula el vecindario inicial. Debe ser la mitad del lado del mapa de Kohonen
vecindario = lado_mapa/2


matriz_pesos = np.random.random((lado_mapa,lado_mapa,dimensiones[0],dimensiones[1]))

## Obtenemos las imagenes

In [3]:

image_list = []#array que contendrá todas las imagenes

#obtenemos las parasitadas
for filename in glob.glob('cell_images\cell_images\Parasitized\*.png'): #obtenemos los nombres de todas las imagenes 
    im=cv.imread(filename, cv.IMREAD_UNCHANGED)/255
    im=cv.resize(im,dimensiones,interpolation=cv.INTER_AREA)
    image_list.append(im)
#obtenemos las no infectadas
for filename in glob.glob(r'cell_images\cell_images\Uninfected\*.png'): #obtenemos los nombres de todas las imagenes
    im=cv.imread(filename, cv.IMREAD_UNCHANGED)/255
    im=cv.resize(im,dimensiones,interpolation=cv.INTER_AREA)
    image_list.append(im)

In [4]:
print(len(image_list))

#utilizaremos esta funcion para mezclar los valores de la lista de imagenes y asi no estén juntas por la clase a la que pertenecen
def mezclar_lista(lista_original):
    lista = lista_original[:]
    longitud_lista = len(lista)

    for i in range(longitud_lista):
        aleatorio = np.random.randint(0, longitud_lista - 1)
        # Intercambiar
        aux = lista[i]
        lista[i] = lista[aleatorio]
        lista[aleatorio] = aux
    # Regresarla
    return lista

27558


In [5]:

lista=mezclar_lista(image_list)
#dividimos en 2 el dataset para valores de entrenamiento y valores de clasificación
x_train, x_resto= train_test_split(image_list, test_size=0.3, shuffle=True)

print(len(x_train), len(x_resto))
    

19290 8268


In [6]:
lado_mapa = 5
periodo = 5000
learning_rate = 0.1

matriz_pesos = np.random.random((lado_mapa,lado_mapa,dimensiones[0],dimensiones[1],3))

In [7]:
len(x_train[0].flatten())

30000

## Definicion de funciones

In [8]:
#Función para calcular la distancia euclidea
def distancia(patron, peso):
    distancia=0
    comp=0
    patron=patron.flatten()
    peso=peso.flatten()
    for i in range(patron.shape[0]):
        #for j in range(patron.shape[1]):
            #for k in range(patron.shape[2]):
        comp+=(peso[i]-patron[i])**2
            #print(i,j)
    distancia=(comp)**0.5
    return distancia

# Función para encontrar la BMU
"""
   Encuentra la BMU para un patrón de entrada.
   Entradas: (patrón_de_entrada, matriz_de_pesos, número_de_entradas)
   Salidas:  (bmu, bmu_idx) tupla donde
               bmu: vector de pesos de la neurona ganadora
               bum_idx: coordenadas de la neurona ganadora
"""
def calcular_bmu(patron_entrada, m_pesos, m):
    MenorDist=0
    MejorVector=[]
    a=[]
    Resultado=0
    idx=[]
    
    for i in range(m):
        for j in range(m):
            
            a=m_pesos[i][j]
            Resultado=distancia(patron_entrada, a)#distancia euclidea
            if i == 0 and j==0 :#salvamos la primera iteracion
                MenorDist=Resultado
                MejorVector=m_pesos[i][j]
                idx=[i,j]
            else:
                #si encuentra una distancia euclidea menor cambiamos
                if MenorDist>Resultado:
                    #print(m_pesos[i][j])
                    MenorDist=Resultado
                    MejorVector=m_pesos[i][j]
                    idx=[i,j]
         
 
    return MejorVector,idx



# Función para calcular el descenso del coeficiente de aprendizaje (eta)
"""
   Calcula el Learning Rate (eta) que corresponde a la i-ésima presentación.
   Entradas: (learning_rate_inicial, iteracion, período)
   Salidas:  learning_rate para la iteración i

"""
def variacion_learning_rate(lr_inicial, i, n_iteraciones):
    
    #lr_final = lr_inicial * (1 - i/n_iteraciones) 
    learning_rate = lr_inicial + (-lr_inicial/n_iteraciones)*i
    return learning_rate

#función para ajustar los pesos durante el entrenamiento
def ajuste_Pesos(i,j,pesos):
    total=0
    for a in range(dimensiones[0]):
        for b in range(dimensiones[1]):
            total+=pesos[i][j][a][b]**2
    return (total)**0.5


# Función para calcular el descenso del vecindario (v)
"""
   Calcula el vecindario  (v) que corresponde a la i-ésima presentación.
   Entradas: (vecindario_inicial, iteracion, período)
   Salidas:  lvecindario para la iteración i

"""
def variacion_vecindario(vecindario_inicial, i, n_iteraciones):
    
    vecindario_i = 1 + vecindario_inicial * (1 - i/n_iteraciones)
    return vecindario_i

# Función para calcular el descenso del coeficiente de aprendizaje (eta) en función de la distancia a la BMU
"""
   Calcula la amortiguación de eta en función de la distancia en el mapa entre una neurona y la BMU.
   Entradas: (distancia_BMU, vecindario_actual)
   Salidas:  amortiguación para la iteración

"""
def decay(distancia_BMU, vecindario_actual):
    return np.exp(-distancia_BMU**2 / (2*vecindario_actual**2))

In [9]:
x_train[5].shape

(100, 100, 3)

In [10]:
#a=[]
#for filename in glob.glob(r'cell_images\cell_images\Parasitized\*.png'): 
#    im=cv.imread(filename, cv.IMREAD_GRAYSCALE)
#    im=cv.resize(im,dimensiones,interpolation=cv.INTER_AREA)
#    a.append(im)
#    break    
#for i in range(100):
#    cv.imshow('imagen',lista[i])
#    cv.waitKey(0)


## Entrenamiento

In [44]:
# Entrena la red con el dataset de entrenamiento

for i in range(len(x_train)):
    azar=np.random.randint(0, len(x_train)) 
    patron=x_train[azar]#patron al azar
    #patron=np.reshape(patron,(dimensiones[0],dimensiones[1]))
    BMU, coordenadas = calcular_bmu(patron, matriz_pesos, lado_mapa)#encontramos la bmu y sus coordenadas
    eta=variacion_learning_rate(learning_rate, i+1, periodo)#calculamos eta
    v=variacion_vecindario(vecindario, i+1, periodo)#calculamos el vecindario
    
    #comprobamos cuales son las neuronas del vecindario de la BMU y les aplicamos el aprendizaje que les corresponde
    for w in range(len(matriz_pesos)):
        for j in range(len(matriz_pesos)):
            deu= ((w-coordenadas[0])**2 + (j-coordenadas[1])**2 )**0.5 #calculamos la distancia del peso a la BMU
            
            if deu <= v :
                amortiguacion= decay(deu, v) #calculamos la amortiguacion
                matriz_pesos[w][j]= matriz_pesos[w][j]/ajuste_Pesos(w,j,matriz_pesos)#ajustamos el peso
                matriz_pesos[w][j]= matriz_pesos[w][j] + eta*amortiguacion*(patron-matriz_pesos[w][j]) #aplicamos aprendizaje
    if i%100==0:
        print("Vuelta nº: ", i)

Vuelta nº:  0


In [21]:
#Función para calcular la distancia euclidea

#Funcion para calcular el error de cuantificacion aplicando su formula
def errorCuantificacion(dis):
    contador = 0
    error_cuantificacion = 0
    
    for j in range(dis.shape[0]):
        for k in range(dis.shape[1]):
            if dis[j][k] != 0:
                contador+=1
                error_cuantificacion += dis[j][k]
                
    error_cuantificacion = error_cuantificacion / contador
    return error_cuantificacion


#Funcion para calcular el error topológico aplicando su formula
def errorTopologico(bmu1, bmu2):
    if ((bmu2[0] == bmu1[0] + 1 or bmu2[0] == bmu1[0] - 1) and bmu2[1] == bmu1[1]) or (bmu2[0] == bmu1[0] and (bmu2[1] == bmu1[1] + 1 or bmu2[1] == bmu1[1] - 1)) or ((bmu2[0] == bmu1[0] - 1) and (bmu2[1] == bmu1[1] + 1 or bmu2[1] == bmu1[1] - 1)) or ((bmu2[0] == bmu1[0] + 1) and (bmu2[1] == bmu1[1] + 1 or bmu2[1] == bmu1[1] - 1)):
        v = 0
    else:
        v = 1
    return v

#Calcular la segunda BMU
def calcular2bmu(patron_entrada, m_pesos, m):
    MenorDist = 999999
    MenorDist2 = 999999
    MejorVector  =  [0,0,0]
    MejorVector2  =  [0,0,0]
    a=[]
    Resultado = 0
    idx=[]
    idx2=[]
    
    for i in range(len(m_pesos[0])):
        for j in range(len(m_pesos[1])):
            a = m_pesos[i][j]
            Resultado = distancia(patron_entrada,a)
            #print(i,j)
            if Resultado< MenorDist:
                #cambiamos la antigua mejor a la segunda mejor
                MenorDist2=MenorDist
                MejorVector2 = MejorVector
                idx2 = idx
                
                #actualizamos la mejor
                MenorDist=Resultado
                MejorVector = m_pesos[i][j]
                idx = [i,j]
                #print(idx,idx2)
                #print(MenorDist, MenorDist2)
            elif Resultado<MenorDist2:
                MenorDist2=Resultado
                MejorVector2 = m_pesos[i][j]
                idx2 = [i,j]

     


    return MenorDist,idx,MejorVector2, idx2



In [22]:
def error_top(bmu1,bmu2):
    v=0
    d=((bmu1[0]-bmu2[0])**2+(bmu1[1]-bmu2[1])**2)**0.5
    
    if d>2**0.5:
        v=1
    else:
        v=0
    return v



In [37]:
#Clasificacion los datos
clasificacion = np.zeros((lado_mapa, lado_mapa, dimensiones[0],dimensiones[1],3))
activacion = np.zeros((lado_mapa, lado_mapa))
distancias = np.zeros((lado_mapa, lado_mapa))
suma = 0
clase = 0
eTopo = 0


for i in range(round(len(x_resto))):
    patron=x_resto[i]#patron al azar
    #patron=np.reshape(patron,(dimensiones[0],dimensiones[1]))
    
    #BMU, coor = calcular_bmu(patron, matriz_pesos, lado_mapa) #BMU
    BMUdist, coor, BMU2, coor2 = calcular2bmu(patron, matriz_pesos, lado_mapa) #BMU2
    
    clasificacion[coor[0]][coor[1]] = patron#asignamos el ultimo patron reconocido por la neurona
    activacion[coor[0]][coor[1]] += 1 #sumamos uno en la neurona activado
    distancias[coor[0]][coor[1]] = distancias[coor[0]][coor[1]] + distancia(patron, BMUdist) #distancia media 
    eTopo += error_top(coor, coor2) # sumamos el error topologico
    if i%100==0:
        print("Vueltas:", i)
    #Calculamos el numero de la clases y guardamos la distancia media de las neuronas asociadas
for g in range(activacion.shape[0]): 
    for t in range(activacion.shape[1]):
        if  activacion[g][t] > 0:
            clase+=1
            distancias[g][t]=distancias[g][t]/activacion[g][t] 
      
            
nclases = clase
mapa_activaciones = activacion
mapa_distancia = distancias
eCuanti = errorCuantificacion(distancias)
eTopo = eTopo/len(x_resto)

print("Mapa Distancias: \n", mapa_distancia)
print("Mapa Activaciones: \n", mapa_activaciones)
print("Clases:", clase)
print("Error Topologico:", eTopo)
print("Error cuantificacion:", eCuanti)




Vueltas: 0


KeyboardInterrupt: 

In [14]:
print("Mapa Distancias: \n", mapa_distancia)
print("Mapa Activaciones: \n", mapa_activaciones)
print("Clases:", clase)
print("Error Topologico:", eTopo)
print("Error cuantificacion:", eCuanti)
lado_mapa

Mapa Distancias: 
 [[ 0.          0.          0.         52.82892575]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
Mapa Activaciones: 
 [[   0.    0.    0. 1378.]
 [   0.    0.    0.    0.]
 [   0.    0.    0.    0.]
 [   0.    0.    0.    0.]]
Clases: 1
Error Topologico: 0.04927788663908847
Error cuantificacion: 52.828925749811816


4